In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
from scipy import optimize

df = pd.read_excel("Exam Data.xlsx", index_col = 0, sheet_name = "Histo underlying")
STOXX_DEC = df.iloc[1:,0]
STOXX_DEC.index.name = "Date"

STOXX = df.iloc[1:,3].dropna()
STOXX.index.name = "Date"

In [82]:
df_call_put = pd.read_excel("Exam Data.xlsx", index_col = 1, sheet_name = "Call-Put Price").drop("CALL", axis = 1).dropna()
df_call = df_call_put.iloc[:9]
df_put = df_call_put.iloc[10:]

In [101]:
df_call

,2021-06-18 00:00:00,2021-12-17 00:00:00,2022-06-17 00:00:00,2022-12-16 00:00:00,2023-12-15 00:00:00,2024-12-20 00:00:00,2025-12-19 00:00:00,2026-12-18 00:00:00
K/T,,,,,,,,
4200,0.00217172,0.0171591,0.0273359,0.0385354,0.0509722,0.06125,0.071,0.0815
4000,0.0132702,0.0370328,0.0463384,0.0580429,0.0691919,0.0784217,0.088,0.094
3800,0.0442045,0.0660985,0.0725758,0.0834091,0.0917551,0.0992298,0.102,0.105
3600,0.0869444,0.102374,0.104747,0.113737,0.118712,0.124533,0.129,0.131
3400,0.134217,0.143434,0.141705,0.148308,0.150492,0.152891,0.155,0.157
3200,0.183283,0.187715,0.182462,0.187134,0.185139,0.184583,0.1836,0.1826
3000,0.23303,0.233889,0.225316,0.228384,0.221995,0.219432,0.218,0.025
2800,0.283068,0.281414,0.270404,0.271919,0.27,0.255795,0.253,0.251
2600,0.325,0.329874,0.315,0.317058,0.315,0.305,0.302,0.3


In [109]:
def BSClosedForm(S_0, K, r, sigma, T, isCall):
        d1 = (np.log(S_0/K) + (r + 0.5 * sigma**2)*T)/(sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if isCall:
            return S_0 * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)
        else:
            return -S_0 * norm.cdf(-d1) + np.exp(-r * T) * K * norm.cdf(-d2)

In [124]:
K, end_date = 3800, pd.datetime(2021, 6, 18)
S_0 = 3956.77
curr_date = pd.datetime(2021, 4, 7)
r = 0.01
isCall = True

def IV(S_0, curr_date, r, K, end_date, isCall):
    T = (end_date - curr_date).days/252
    if isCall:
        p = df_call.loc[K,end_date] * S_0
    else:
        p = df_put.loc[K,end_date] * S_0
    a, b = 0.000001, 1
    f = lambda x: BSClosedForm(S_0, K, r, x, T, isCall) - p
    sol = optimize.brentq(f, a, b)
    return sol

In [125]:
IV(S_0, curr_date, r, K, T, isCall)

0.06611929466782246